In [1]:
# Original implementation: https://www.geeksforgeeks.org/opencv-real-time-road-lane-detection/

In [2]:
%matplotlib inline
import os
import urllib
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML

VIDEO_URL = 'https://raw.githubusercontent.com/rslim087a/road-video/master/test2.mp4'
VIDEO_DIRECTORY = './'
VIDEO_FILENAME = os.path.join(VIDEO_DIRECTORY, VIDEO_URL.rsplit('/', 1)[-1])

In [3]:
# Download video

import urllib

def download_video():
    
    if not os.path.isfile(VIDEO_FILENAME):
        urllib.request.urlretrieve(VIDEO_URL, VIDEO_FILENAME)
        
download_video()

In [4]:
# Canny edge detection

def canny_edge_detector(image):
    # Convert the image color to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)  
    
    # Reduce noise from the image
    blur = cv2.GaussianBlur(gray_image, (5, 5), 0)  
    canny = cv2.Canny(blur, 50, 150) 
    return canny

In [5]:
# ROI

def region_of_interest(image): 
    height = image.shape[0] 
    polygons = np.array([ 
        [(200, height), (1100, height), (550, 250)] 
        ]) 
    mask = np.zeros_like(image) 
      
    # Fill poly-function deals with multiple polygon 
    cv2.fillPoly(mask, polygons, 255)  
      
    # Bitwise operation between canny image and mask image 
    masked_image = cv2.bitwise_and(image, mask)  
    return masked_image

In [6]:
# Find coordinates of road land

def create_coordinates(image, line_parameters): 
    slope, intercept = line_parameters 
    y1 = image.shape[0] 
    y2 = int(y1 * (3 / 5)) 
    x1 = int((y1 - intercept) / slope) 
    x2 = int((y2 - intercept) / slope) 
    return np.array([x1, y1, x2, y2]) 

In [7]:
# Average slope intercept

def average_slope_intercept(image, lines): 
    left_fit = [] 
    right_fit = [] 
    for line in lines: 
        x1, y1, x2, y2 = line.reshape(4) 
          
        # It will fit the polynomial and the intercept and slope 
        parameters = np.polyfit((x1, x2), (y1, y2), 1)  
        slope = parameters[0] 
        intercept = parameters[1] 
        if slope < 0: 
            left_fit.append((slope, intercept)) 
        else: 
            right_fit.append((slope, intercept)) 
              
    left_fit_average = np.average(left_fit, axis = 0) 
    right_fit_average = np.average(right_fit, axis = 0) 
    left_line = create_coordinates(image, left_fit_average) 
    right_line = create_coordinates(image, right_fit_average) 
    return np.array([left_line, right_line]) 

In [8]:
# Display lines

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

In [9]:
# Path of dataset directory 
cap = cv2.VideoCapture(VIDEO_FILENAME)  

In [10]:
frames = []
combos = []

while(cap.isOpened()):
        
    _, frame = cap.read()
    
    try:
        canny_image = canny_edge_detector(frame)
        cropped_image = region_of_interest(canny_image)

        lines = cv2.HoughLinesP(cropped_image, 2, np.pi / 180, 100,
                                np.array([]), minLineLength = 40,
                                maxLineGap = 5)

        averaged_lines = average_slope_intercept(frame, lines)
        line_image = display_lines(frame, averaged_lines)
        combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
        frames.append(frame)
        combos.append(combo_image)
        
    except:
        break
        
cap.release()

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [11]:
def plot_sequence_images(image_array, ):
    ''' Display images sequence as an animation in jupyter notebook
    
    Args:
        image_array(numpy.ndarray): image_array.shape equal to (num_images, height, width, num_channels)
    '''
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[:2]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        tmp = np.zeros(image_array[i].shape, dtype=np.uint8)
        tmp[:, :, 0] = image_array[i][:, :, 2]
        tmp[:, :, 1] = image_array[i][:, :, 1]
        tmp[:, :, 2] = image_array[i][:, :, 0]
        im.set_array(tmp)
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array), interval=33, repeat_delay=1, repeat=True)
    display(HTML(anim.to_html5_video()))

In [14]:
plot_sequence_images(frames)

<Figure size 1280x720 with 0 Axes>

In [15]:
plot_sequence_images(combos)

<Figure size 1280x720 with 0 Axes>